# Bitcoin - Demo Breakdown


* Step 1: SHA-256 Hashing: Demonstrate cryptographic hashing.
* Step 2: Creating Transactions: Build simple transactions.
* Step 3: Building Blocks: Create blocks that contain transactions and link them.
* Step 4: Proof of Work (Mining): Simulate mining by finding a valid hash.
* Step 5: Constructing the Blockchain: Chain the blocks together and ensure validity.
* Conclusion: Final demonstration of the blockchain and a note on immutability and security.

In [1]:
pip install ecdsa base58

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import ecdsa
import base58
import hashlib

## Step 1: Generate a random private key, and derive the public key, and address from it 
Using ecdsa and hashlib libraries, we can generate a random private key, and derive the public key, and address from it.

In [3]:
# Generate a private key
private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
private_key_hex = private_key.to_string().hex()
print("Private Key (hex):", private_key_hex)

# Derive the public key from the private key
public_key = private_key.get_verifying_key()
public_key_hex = '04'+ public_key.to_string().hex()
print("Public Key (hex):", public_key_hex)

#generate bitcoin address from public key p2pkh
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(hashlib.sha256(bytes.fromhex(public_key_hex)).digest())
public_key_hash = ripemd160.digest()
print("Public Key Hash:", public_key_hash.hex())

#add version byte
version = b'\x00'
public_key_hash_with_version = version + public_key_hash
print("Public Key Hash with Version:", public_key_hash_with_version.hex())

#calculate checksum
checksum = hashlib.sha256(hashlib.sha256(public_key_hash_with_version).digest()).digest()[:4]
print("Checksum:", checksum.hex())

#append checksum to public key hash with version
address = public_key_hash_with_version + checksum

#base58 check encode
address = base58.b58encode(address)
print("Base58 Check Encoded Address:", address.decode())

Private Key (hex): c98ad27f96907a9ca818c4c8c1e3ee588c3faf7cf7ef3bb600e0360b428f2341
Public Key (hex): 048871e6491ad35921e61c4ab271cf1057aaf74327236055eb5d527e8427ef5513e1205e318571ccc72e6c4c5231dcd60bb5894ddd47c0a9805ff258f1ad0b369d
Public Key Hash: a07cb29a559df2b57928f257460c6d7c426123f0
Public Key Hash with Version: 00a07cb29a559df2b57928f257460c6d7c426123f0
Checksum: f5c02cea
Base58 Check Encoded Address: 1FdaVqeNQMNtByAPkpCpMbFwK4q81XZW1w


## Step 2: Create a UTXO (Unspent Transaction Output) 


In [9]:
class Wallet:
    def __init__(self):
        # Generate private and public keys upon wallet creation
        self.private_key = self.generate_private_key()
        self.public_key = self.generate_public_key()
        self.address = self.generate_address()
        self.utxos = []  # Store UTXOs related to this wallet

    def generate_private_key(self):
        # Generate a private key using secp256k1 elliptic curve
        private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        return private_key
    
    def generate_public_key(self):
        # Derive the public key from the private key
        public_key = self.private_key.get_verifying_key()
        # Return uncompressed public key with '04' prefix
        return '04' + public_key.to_string().hex()
    
    def generate_address(self):
        # Generate a Bitcoin address from the public key (P2PKH)
        ripemd160 = hashlib.new('ripemd160')
        ripemd160.update(hashlib.sha256(bytes.fromhex(self.public_key)).digest())
        public_key_hash = ripemd160.digest()
        
        # Add the version byte (0x00 for mainnet)
        version = b'\x00'
        public_key_hash_with_version = version + public_key_hash
        
        # Compute checksum (double SHA-256)
        checksum = hashlib.sha256(hashlib.sha256(public_key_hash_with_version).digest()).digest()[:4]
        
        # Append the checksum to the public key hash with version byte
        address_binary = public_key_hash_with_version + checksum
        
        # Return Base58Check encoded address
        return base58.b58encode(address_binary).decode()

    def get_balance(self):
        # Calculate the balance by summing the amounts in UTXOs
        balance = sum(utxo.amount for utxo in self.utxos)
        return balance

    def add_utxo(self, utxo):
        # Add a new UTXO to the wallet
        self.utxos.append(utxo)
    
    def create_transaction(self, amount, recipient_address, fee=0.001):
        # Create a transaction using available UTXOs
        total_required = amount + fee
        if self.get_balance() < total_required:
            raise Exception("Not enough balance")

        # Find UTXOs to cover the amount + fee
        inputs = []
        total_input = 0
        for utxo in self.utxos:
            inputs.append(utxo)
            total_input += utxo.amount
            if total_input >= total_required:
                break
    
        # Remove used UTXOs from the wallet
        for utxo in inputs:
            self.utxos.remove(utxo)

        # Create transaction outputs
        outputs = [TransactionOutput(amount, recipient_address)]
        change = total_input - total_required
        if change > 0:
            # Send the remaining balance (change) back to the wallet
            change_output = TransactionOutput(change, self.address)
            outputs.append(change_output)

            # Add the change UTXO back to the wallet
            new_change_utxo = UTXO(txid="new_txid", index=1, amount=change, address=self.address)
            self.utxos.append(new_change_utxo)

        # Create a transaction object
        transaction = Transaction(inputs, outputs)
    
        # Sign the transaction using the wallet's private key
        transaction.sign_transaction(self.private_key)
        return transaction


    def __repr__(self):
        return f"Wallet(Address: {self.address}, Balance: {self.get_balance()} BTC)"


In [5]:
class UTXO:
    def __init__(self, txid, index, amount, address):
        self.txid = txid       # The transaction ID of the UTXO
        self.index = index     # Output index in the transaction
        self.amount = amount   # Amount of Bitcoin in this UTXO
        self.address = address # The address that can spend this UTXO

    def __repr__(self):
        return f"UTXO(txid={self.txid}, index={self.index}, amount={self.amount}, address={self.address})"


In [6]:
class TransactionInput:
    def __init__(self, utxo):
        self.txid = utxo.txid    # The transaction ID of the UTXO being spent
        self.index = utxo.index  # The index of the UTXO in the previous transaction
        self.amount = utxo.amount # The amount of Bitcoin in the UTXO
        self.signature = None     # Signature will be added once signed
    
    def __repr__(self):
        return f"TransactionInput(txid={self.txid}, index={self.index}, amount={self.amount}, signature={self.signature})"

class TransactionOutput:
    def __init__(self, amount, address):
        self.amount = amount    # Amount of Bitcoin to be sent
        self.address = address  # Receiver's Bitcoin address
    
    def __repr__(self):
        return f"TransactionOutput(amount={self.amount}, address={self.address})"

class Transaction:
    def __init__(self, inputs, outputs):
        self.inputs = inputs      # List of TransactionInput objects
        self.outputs = outputs    # List of TransactionOutput objects
        self.txid = self.calculate_txid()  # Transaction ID based on inputs and outputs

    def calculate_txid(self):
        # Hash of transaction data to generate a transaction ID
        tx_data = str(self.inputs) + str(self.outputs)
        return hashlib.sha256(tx_data.encode()).hexdigest()

    def sign_transaction(self, private_key):
        # Sign each input with the private key of the wallet
        for tx_input in self.inputs:
            message = f"{tx_input.txid}:{tx_input.index}:{tx_input.amount}".encode()
            signature = private_key.sign(message)
            tx_input.signature = signature.hex()

    def verify_transaction(self, public_key):
        # Verify the signature for each input using the public key
        for tx_input in self.inputs:
            message = f"{tx_input.txid}:{tx_input.index}:{tx_input.amount}".encode()
            signature = bytes.fromhex(tx_input.signature)
            if not public_key.verify(signature, message):
                return False  # Invalid transaction if any signature fails
        return True  # Valid transaction if all signatures are correct

    def __repr__(self):
        return f"Transaction(txid={self.txid}, inputs={self.inputs}, outputs={self.outputs})"

In [23]:
import time
import hashlib

class Block:
    def __init__(self, index, previous_hash, transactions, difficulty=2):
        self.index = index
        self.timestamp = time.time()
        self.transactions = transactions  # List of transactions in the block
        self.previous_hash = previous_hash  # Hash of the previous block
        self.nonce = 0
        self.difficulty = difficulty  # Number of leading zeros required in the hash
        self.hash = self.mine_block()  # Call mine_block to generate and set the hash

    def calculate_hash(self):
        # Hash the block’s content (transactions, previous hash, nonce)
        block_data = str(self.index) + str(self.timestamp) + str(self.transactions) + self.previous_hash + str(self.nonce)
        return hashlib.sha256(block_data.encode()).hexdigest()

    def mine_block(self):
        target = '0' * self.difficulty  # Hash must start with this many zeros
        calculated_hash = self.calculate_hash()  # Start calculating the hash
        while calculated_hash[:self.difficulty] != target:
            self.nonce += 1
            calculated_hash = self.calculate_hash()
        return calculated_hash  # Set the final mined hash after PoW is done

    def __repr__(self):
        return f"Block(index={self.index}, hash={self.hash}, previous_hash={self.previous_hash}, nonce={self.nonce})"


class Blockchain:
    def __init__(self):
        genesis_block = Block(0, "0" * 64, [], difficulty=2)
        self.chain = [genesis_block]

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, transactions):
        previous_hash = self.get_latest_block().hash
        new_block = Block(index=len(self.chain), previous_hash=previous_hash, transactions=transactions)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True


In [24]:
class Script:
    def __init__(self, public_key_hash):
        # Locking script (scriptPubKey): requires signature and pubkey to unlock
        self.scriptPubKey = f"OP_DUP OP_HASH160 {public_key_hash} OP_EQUALVERIFY OP_CHECKSIG"
    
    def validate(self, public_key, signature):
        # Simulate script validation (simplified)
        return hashlib.sha256(public_key.encode()).hexdigest() == signature



In [25]:
# Create a new wallet
wallet = Wallet()

# Add two UTXOs (Simulating receiving 10 BTC and 5 BTC)
wallet.add_utxo(UTXO("txid1", 0, 10, wallet.address))  # 10 BTC UTXO
wallet.add_utxo(UTXO("txid2", 1, 5, wallet.address))   # 5 BTC UTXO

# Display wallet information
print(f"Generated Wallet Address: {wallet.address}")
print(f"Wallet Balance: {wallet.get_balance()} BTC")


Generated Wallet Address: 1BSVH3RHPjJ8fBQXzyoMYr2er7bMRkEK7q
Wallet Balance: 15 BTC


In [26]:
# Create a transaction to send 12 BTC to a recipient with a 0.001 BTC fee
transaction = wallet.create_transaction(12, "1RecipientAddress", fee=0.001)

# Display the created transaction
print("Created Transaction:")
print(transaction)

# Verify the transaction (using the wallet's public key)
is_valid = transaction.verify_transaction(wallet.private_key.get_verifying_key())
print(f"Is the transaction valid? {is_valid}")

# Display wallet balance after creating the transaction
print(f"Wallet Balance after transaction: {wallet.get_balance()} BTC")


Created Transaction:
Transaction(txid=f1809de414c5aeaca7b29fa96cb3d3480bc27c839f58099d9a3b32ca0eba2868, inputs=[UTXO(txid=txid1, index=0, amount=10, address=1BSVH3RHPjJ8fBQXzyoMYr2er7bMRkEK7q), UTXO(txid=txid2, index=1, amount=5, address=1BSVH3RHPjJ8fBQXzyoMYr2er7bMRkEK7q)], outputs=[TransactionOutput(amount=12, address=1RecipientAddress), TransactionOutput(amount=2.9990000000000006, address=1BSVH3RHPjJ8fBQXzyoMYr2er7bMRkEK7q)])
Is the transaction valid? True
Wallet Balance after transaction: 2.9990000000000006 BTC


In [27]:
# Create a block with the transaction and mine it
transactions = [transaction]
block = Block(index=1, previous_hash="0" * 64, transactions=transactions, difficulty=3)

# Display the mined block
print("Mined Block:")
print(block)


Mined Block:
Block(index=1, hash=000ab5918febe939d517cb51fd2ed57aa7ac644480a05349241d2a9cd0adfbed, previous_hash=0000000000000000000000000000000000000000000000000000000000000000, nonce=1297)


In [28]:
# Create a new blockchain
blockchain = Blockchain()

# Add the mined block to the blockchain
blockchain.add_block(transactions)

# Verify the integrity of the blockchain
is_chain_valid = blockchain.is_chain_valid()
print(f"Is the blockchain valid? {is_chain_valid}")

# Display the blockchain
print("Blockchain:")
for block in blockchain.chain:
    print(block)


Is the blockchain valid? True
Blockchain:
Block(index=0, hash=00535f29c034c4f938b44cbc5dc1f990c59ef9b9daa155f335958155c7e833ec, previous_hash=0000000000000000000000000000000000000000000000000000000000000000, nonce=770)
Block(index=1, hash=002de4505a60275a21343e483e6a94c39d140b3725af31991302d44075a601cd, previous_hash=00535f29c034c4f938b44cbc5dc1f990c59ef9b9daa155f335958155c7e833ec, nonce=209)
